In [32]:
import torchvision
import pandas as pd
import numpy as np
from tensorboardX import SummaryWriter
from helper import plot_results, predict, train, split_data

In [33]:
writer = SummaryWriter('runs1/msft_stock_experiment_backup')

In [34]:
df = pd.read_csv("MSFT.csv", parse_dates=True, usecols=['Date', 'Close'])

In [35]:
# Plot data
import plotly.express as px

fig = px.line(df, x='Date', y='Close', title='Time Series with Range Slider and Selectors')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

## Data Cleaning / Preprocessing

In [36]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

def preprocess_data(df):
    df_msft = df[['Close']]
    df_msft = df_msft.fillna(method='ffill')
    df_msft['Close'] = scaler.fit_transform(df_msft['Close'].values.reshape(-1,1))
    return df_msft

df_msft = preprocess_data(df)

In [37]:
import torch
import torch.nn as nn

lookback = 100
x_train, y_train, x_test, y_test = split_data(df_msft, lookback)

In [38]:
# Model params
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 25

## LSTM Model

In [39]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [40]:
model_lstm = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

writer.add_graph(model_lstm, x_train)

### LSTM Training

In [41]:
train(num_epochs, model_lstm, x_train, y_train, writer, 'LSTM Training Loss')

Epoch  0 MSE:  0.34765616059303284
Epoch  1 MSE:  0.22341081500053406
Epoch  2 MSE:  0.12341698259115219
Epoch  3 MSE:  0.10695619136095047
Epoch  4 MSE:  0.10820438712835312
Epoch  5 MSE:  0.08281915634870529
Epoch  6 MSE:  0.08059047162532806
Epoch  7 MSE:  0.08154038339853287
Epoch  8 MSE:  0.0752328634262085
Epoch  9 MSE:  0.06048634648323059
Epoch  10 MSE:  0.04355834797024727
Epoch  11 MSE:  0.04169035330414772
Epoch  12 MSE:  0.033447399735450745
Epoch  13 MSE:  0.013343552127480507
Epoch  14 MSE:  0.017251815646886826
Epoch  15 MSE:  0.014747589826583862
Epoch  16 MSE:  0.008733440190553665
Epoch  17 MSE:  0.01939566433429718
Epoch  18 MSE:  0.007316725794225931
Epoch  19 MSE:  0.007030607201159
Epoch  20 MSE:  0.00719024334102869
Epoch  21 MSE:  0.004031236749142408
Epoch  22 MSE:  0.0027367512229830027
Epoch  23 MSE:  0.004759977571666241
Epoch  24 MSE:  0.006289965007454157


### LSTM Prediction

In [42]:
y_test_lstm, y_test_pred_lstm = predict(model_lstm, x_test, y_test, scaler)

plt, figure = plot_results(y_test_lstm, y_test_pred_lstm, df)
writer.add_figure("LSTM Prediction", figure)

plt.show()

## GRU Model

In [43]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        # out = self.relu(out[:, -1, :])
        out = self.fc(out[:, -1, :])
        return out

In [44]:
model_gru = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

writer.add_graph(model_gru, x_train)

### GRU Training

In [45]:
train(num_epochs, model_gru, x_train, y_train, writer, 'GRU Training Loss')

Epoch  0 MSE:  0.39186373353004456
Epoch  1 MSE:  0.18085570633411407
Epoch  2 MSE:  0.07534751296043396
Epoch  3 MSE:  0.1293610781431198
Epoch  4 MSE:  0.08869601041078568
Epoch  5 MSE:  0.05188676342368126
Epoch  6 MSE:  0.04926262050867081
Epoch  7 MSE:  0.053009431809186935
Epoch  8 MSE:  0.04640667513012886
Epoch  9 MSE:  0.027779342606663704
Epoch  10 MSE:  0.007831679657101631
Epoch  11 MSE:  0.00806950218975544
Epoch  12 MSE:  0.02268308773636818
Epoch  13 MSE:  0.012254258617758751
Epoch  14 MSE:  0.004031167831271887
Epoch  15 MSE:  0.010158592835068703
Epoch  16 MSE:  0.014322038739919662
Epoch  17 MSE:  0.010547826997935772
Epoch  18 MSE:  0.004124891944229603
Epoch  19 MSE:  0.0011598410783335567
Epoch  20 MSE:  0.0033978689461946487
Epoch  21 MSE:  0.0071957712061703205
Epoch  22 MSE:  0.007966138422489166
Epoch  23 MSE:  0.005581878591328859
Epoch  24 MSE:  0.003175577614456415


### GRU Prediction

In [46]:
y_test_gru, y_test_pred_gru = predict(model_gru, x_test, y_test, scaler)

plt, figure = plot_results(y_test_gru, y_test_pred_gru, df)
writer.add_figure("GRU Prediction", figure)

plt.show()